# 데이터 분석7. 텍스트 데이터 전처리

## Ⅰ 텍스트 전처리란?

텍스트 데이터 전처리는 인공지능과 데이터 분석에서 필수적인 단계로, 데이터의 품질을 높여 모델의 성능을 극대화한다. 이 과정에는 불필요한 문자나 단어 제거, 단어의 형태 통일, 형태소 분석 등이 포함된다. 그리고 이를 통해 데이터 품질을 향상시키고, 자연어처리 작업의 정확도와 효율성을 높이며 워드클라우드와 같은 시각화로도 활용될 수 있다.

전처리에는 정제(Cleaning), 정규화(Normalization), 정규식(Regular Expression), 불용어 제거, 형태소 분석 등이 포함된다.

## ⅠⅠ 개념과 연습

## 1. 텍스트 정제(Cleaning)
텍스트 정제는 원시 텍스트에서 분석에 방해가 되는 불필요한 부분(노이즈)를 제거하는 작업이며 다음과 같은 작업들이 포함된다.

- 특수문자, HTML 태그, 불필요한 공백, 숫자 등 의미 없는 요소들을 제거하는 작업. 

- 등장 빈도가 매우 낮거나 너무 짧은은 단어 제거
  *한국어의 특성상 한 글자 단어가 많으므로 단순히 길이를 기준으로 제거하는 데는 주의 필요

- 의미가 거의 없고 자주 등장하는 조사, 접속사 등 불용어(stopwords)를 제거

### 1-1. 노이즈 제거 함수 (특수문자, 숫자 제거)

In [1]:
import re
from collections import Counter

def clean_text(text):
    cleaned = re.sub(r'[^가-힣a-zA-Z\s]', '', text)
    return cleaned
    # 한글과 영어, 공백만 남기기
    # ^ : 대괄호 안에서 맨 앞에 쓰이면 부정(not) 의미. 즉, 뒤에 나오는 문자들을 제외한 나머지.
    # 가-힣 : 한글 음절 범위를 의미해. 한글의 모든 완성형 글자들이 이 범위에 포함됨.
    # a-z : 영어 소문자 a부터 z까지
    # A-Z : 영어 대문자 A부터 Z까지
    # \s : 공백 문자(스페이스, 탭, 줄바꿈 등)

sample_text = "안녕하세요! This is a test... 전화번호는 010-1234-5678 입니다."
print("원본:", sample_text)
print("정제 후:", clean_text(sample_text))

원본: 안녕하세요! This is a test... 전화번호는 010-1234-5678 입니다.
정제 후: 안녕하세요 This is a test 전화번호는  입니다


### 1-2. 등장 빈도 적은 단어 제거

In [2]:
def remove_rare_words(tokens, min_freq=2):
    freq = Counter(tokens)
    filtered = [w for w in tokens if freq[w] >= min_freq]
    return filtered

tokens = ["서울", "서울", "부산", "대구", "서울", "광주", "대구", "포항"]
print("원본 토큰:", tokens)
print("빈도 2 이상 단어만:", remove_rare_words(tokens, min_freq=2))

원본 토큰: ['서울', '서울', '부산', '대구', '서울', '광주', '대구', '포항']
빈도 2 이상 단어만: ['서울', '서울', '대구', '서울', '대구']


### * 한국어에서 짧은 단어 제거 주의점  
한 글자 단어가 의미 있는 경우가 많으므로 무조건 제거하면 중요한 단어가 사라진다.  
ex) '나', '너', '가' 등

### 1-3. 불용어 제거

In [3]:
stopwords = ['의', '가', '이', '은', '들', '는', '을', '를', '에', '와', '과', '도']

def remove_stopwords(tokens):
    return [w for w in tokens if w not in stopwords]

sample_tokens = ["나는", "오늘", "서울", "에", "갔습니다", "그리고", "친구", "와", "만났습니다"]
print("불용어 제거 전:", sample_tokens)
print("불용어 제거 후:", remove_stopwords(sample_tokens))

불용어 제거 전: ['나는', '오늘', '서울', '에', '갔습니다', '그리고', '친구', '와', '만났습니다']
불용어 제거 후: ['나는', '오늘', '서울', '갔습니다', '그리고', '친구', '만났습니다']


## 2. 텍스트 정규화(Normalization)
정규화는 다양한 표현으로 나타나는 단어를 통일하는 작업이다. 


- '서울', '서울시', '서울특별시'를 모두 '서울'로 변경 
- 대소문자를 일괄적으로 맞추는 작업
-  표제어 추출(lemmatization, 사전에 등재된 기본형으으로 변환)과 어간 추출(stemming, 단어의 접사를 잘라내어 어간만 남김)

### 2-1.규칙 기반 정규화

In [4]:
def normalize_word(word):
    mapping = {
        "US": "USA",
        "서울시": "서울",
        "서울특별시": "서울"
    }
    return mapping.get(word, word)

words = ["US", "USA", "서울시", "서울특별시", "부산"]
normalized_words = [normalize_word(w) for w in words]
print("규칙 기반 정규화:", normalized_words)

규칙 기반 정규화: ['USA', 'USA', '서울', '서울', '부산']


In [5]:
# 문장 간단화 예시
sentence = "나는 서울특별시에 살고 있고, 그렇지만 US를 매우 좋아합니다."
# 단순 띄어쓰기 기준 분리 (실제 NLP에서는 형태소 분석 필요)
sentence_words = sentence.replace(",", "").replace(".", "").split()
normalized_sentence_words = [normalize_word(w) for w in sentence_words]
normalized_sentence = ' '.join(normalized_sentence_words)
print("정규화 전:", sentence)
print("정규화 후:", normalized_sentence)

정규화 전: 나는 서울특별시에 살고 있고, 그렇지만 US를 매우 좋아합니다.
정규화 후: 나는 서울특별시에 살고 있고 그렇지만 US를 매우 좋아합니다


띄어쓰기가 안 된 단어들은 정규화 인식을 하지 못한다. 실제 NLP에서는 형태소 분석이 필요하지만, 지금은 더욱 정밀한 정규화를 시도해본다.

In [6]:
def normalize_word(word):
    mapping = {
        "US": "USA",
        "서울시": "서울",
        "서울특별시": "서울"
    }
    for k, v in mapping.items():
        word = word.replace(k, v)
    return word
# mapping.items()는 딕셔너리의 (키, 값) 쌍을 하나씩 꺼낸다. 
# word.replace(k, v)는 word 문자열에서 k를 찾아 모두 v로 바꾼다. (실제 NLP에서는 형태소 분석 필요)

sentence = "나는 서울특별시에 살고 있고, 그렇지만 US를 매우 좋아합니다."
sentence_words = sentence.replace(",", "").replace(".", "").split()
normalized_sentence_words = [normalize_word(w) for w in sentence_words]
normalized_sentence = ' '.join(normalized_sentence_words)
print("정규화 전:", sentence)
print("정규화 후:", normalized_sentence)

정규화 전: 나는 서울특별시에 살고 있고, 그렇지만 US를 매우 좋아합니다.
정규화 후: 나는 서울에 살고 있고 그렇지만 USA를 매우 좋아합니다


### 2-2. 대소문자 통합

In [7]:
sample_english = ["Apple", "apple", "APPLE", "Banana"]
lowercase_words = [w.lower() for w in sample_english]
print("대소문자 통합:", lowercase_words)

대소문자 통합: ['apple', 'apple', 'apple', 'banana']


### 2-3. Lemmatization vs Stemming 


In [8]:
!pip install nltk

In [9]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

from nltk.stem import PorterStemmer, WordNetLemmatizer

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

words = ["am", "are", "is", "having", "runs"]

print("\n표제어 추출 결과:")
for w in words:
    print(f"{w} -> {lemmatizer.lemmatize(w, pos='v')}")

print("어간 추출 결과:")
for w in words:
    print(f"{w} -> {ps.stem(w)}")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\EL76\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\EL76\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



표제어 추출 결과:
am -> be
are -> be
is -> be
having -> have
runs -> run
어간 추출 결과:
am -> am
are -> are
is -> is
having -> have
runs -> run


## 3. 정규식(Regular Expression)을 이용한 패턴 제거
정규식은 텍스트에서 특정한 패턴을 찾아내거나 제거하는 데 유용하다. 예를 들어, 전화번호 형식, 이메일 주소, HTML 태그 등을 정규식으로 필터링하여 텍스트를 더 깔끔하게 다듬을 수 있다.

파이썬의 re 라이브러리는 이러한 정규식을 지원하며, 복잡한 텍스트 전처리에 자주 사용된다.

### *정규화와 정규식의 관계
- 정규식은 정규화 작업을 수행하는 데 자주 활용된다.
 예를 들어, 정규식을 사용해 문장에서 '서울시'나 '서울특별시'라는 패턴을 찾아 모두 '서울'로 바꾼다면, 이 작업은 정규식을 활용한 정규화라고 할 수 있다

- 한편, 정규화는 정규식 이외에에 단순 치환, 소문자 변환, 표제어 추출 등 다양한 방법으로 이뤄질 수 있다. 

- 물론 정규식은 정규화 이외의 데이터 추출과 분리 등의 작업에도 이용될 수 있다. 즉 **정규화**는 '표현 통일'이라는 *목적*에 해당하고, **정규식**은 '패턴 탐색 및 변환'이라는 *도구*에 해당한다.

### 3-1. XML, HTML 등의 태그 제거

In [10]:
def remove_html_tags(text):
    # Non-greedy 매칭으로 태그 제거
    cleaned = re.sub(r'<.*?>', '', text, flags=re.DOTALL)
    return cleaned

html_text = """
<html>
<head><title>테스트</title></head>
<body><p>안녕하세요!</p><br/><a href="link">링크</a></body>
</html>
"""

print("원본 HTML:", html_text)
print("태그 제거 후:", remove_html_tags(html_text))

원본 HTML: 
<html>
<head><title>테스트</title></head>
<body><p>안녕하세요!</p><br/><a href="link">링크</a></body>
</html>

태그 제거 후: 

테스트
안녕하세요!링크




### 3-2. 다양한 전화번호 형식을 xxx-xxxx-xxxx로 바꾸기

In [11]:
def normalize_phone_numbers(text):
    # 다양한 형태의 전화번호 패턴을 그룹으로 잡아서 -로 구분된 형태로 변환
    pattern = r'(01[016789])[ -]?(\d{3,4})[ -]?(\d{4})'
    normalized = re.sub(pattern, r'\1-\2-\3', text)
    return normalized

sample_text = "연락처: 01012345678, 010-1234-5678, 010 1234 5678"

print("원본 텍스트:", sample_text)
print("정규화 후:", normalize_phone_numbers(sample_text))

원본 텍스트: 연락처: 01012345678, 010-1234-5678, 010 1234 5678
정규화 후: 연락처: 010-1234-5678, 010-1234-5678, 010-1234-5678


### 3-3. Greedy, Non-Greedy (Back-tracking)

- Greedy (탐욕적 방식): 정규 표현식에서 가능한 가장 긴 문자열을 매칭하려는 방식. 즉, 일치하는 문자열 중에서 최대한 많은 문자를 포함하려고 시도
  - 정규식: a.*a
      - . : 아무 문자 1개
      - * : 앞의 문자가 0개 이상 반복 (Greedy하게 최대한 반복)
- Non-Greedy (Lazy): 가능한 가장 짧은 문자열만 매칭하려고 하는 방식. ?를 추가하여 나타냄

In [12]:
print(re.match(r"a*a", "aaba").group())     # Greedy1: a를 0번 이상 반복 + 그 다음 'a'들들(이후에는b이므로 표기x)
print(re.match(r"a.*a", "aaba").group())    # Greedy2: a를 0번 이상 반복 + 그 다음 'a'까지의 모든 문자

print(re.match(r"a*?a", "aaba").group())    # Non-Greedy
print(re.match(r"a.*?a", "aaba").group())   # Non-Greedy
print(re.match(r"a*+a", "aaba").group())    # Possessive quantifier (파이썬 re에서는 지원 X)

aa
aaba
a
aa


AttributeError: 'NoneType' object has no attribute 'group'

### 3-4. (?:...) 표현식

In [13]:
match = re.match(r'(hello) (\w+)', 'hello world')
print(match.group(1))  # hello (저장됨)
print(match.group(2))  # world

hello
world


In [14]:
match = re.match(r'(?:hello) (\w+)', 'hello world')

# match.group(1)은 이제 (\w+)에 대응되며 (?:hello)는 저장하지 않음
print(match.group(1))  # world

world


## 4. 한국어 형태소 분석
한국어는 영어와 달리 조사, 어미 등이 붙어 단어의 형태가 다양하게 변하기 때문에, 형태소 분석기가 꼭 필요하다. 형태소 분석기는 텍스트를 형태소 단위로 분리하고 품사 태깅을 수행한다.

주요 한국어 형태소 분석기에는 Okt, Mecab, Komoran, Kkma, Hannanum 등이 있으며, 파이썬에서는 KoNLPy 라이브러리를 통해 이들을 활용할 수 있다.

예를 들어 Okt는 간단한 문장 형태소 분석에 적합하며, 아래와 같이 사용할 수 있다.

In [15]:
!pip install konlpy

In [16]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [17]:
from konlpy.tag import Okt
okt = Okt()

text = "이것도 되나욬ㅋㅋ"

# 1) 기본 품사 태깅 (norm, stem 없음)
print(okt.pos(text))
# 결과 예:
# [('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나욬', 'Noun'), ('ㅋㅋ', 'KoreanParticle')]

# 2) 정규화 적용 (norm=True)
print(okt.pos(text, norm=True))
# [('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나요', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]

# 3) 정규화 + 어간 추출 적용 (norm=True, stem=True)
print(okt.pos(text, norm=True, stem=True))
# [('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]


JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

In [ ]:
C:\Users\EL76\OneDrive\Desktop\IT\jdk-24_windows-x64_bin\jdk-24.0.1

{'terminal.integrated.env.windows': {'JAVA_HOME': 'C:\\Program Files\\Java\\jdk-XX.X.X',
  'Path': '%JAVA_HOME%\\bin;%Path%'}}